In [ ]:
import json
import os
import random
import re
import time
from pathlib import Path
from typing import cast

import faiss
import numpy as np
import owlready2
import owlrl
import requests
from conceptnet5 import uri as cn_uri, nodes as cn_nodes, api as cn_api
from conceptnet5.nodes import standardized_concept_uri
from pydantic import BaseModel
from rdflib import Graph, URIRef, DCTERMS, RDFS, Literal, Namespace, RDF, OWL, SKOS, BNode
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Whether to check if a DBPedia link exists for a given concept
DONT_CHECK_FOR_URI_EXISTS = False

In [ ]:
# Load the output from the research paper information extraction pipeline
file = Path(os.getcwd()).joinpath("final_concepts.csv")
results_file = open(file)
results_file_contents: str = results_file.read()

In [ ]:
paper_dois_and_titles = [
    ["10.3233/FAIA240179", "https://doi.org/10.3233/FAIA240179",
     "Formal Specification of Actual Trust in Multiagent Systems"],
    ["10.3233/FAIA240201", "https://doi.org/10.3233/FAIA240201",
     "Common Ground Provides a Mental Shortcut in Agent-Agent Interaction"],
    ["hhai-2021_paper_46.pdf",
     "https://www.hhai-conference.org/wp-content/uploads/2022/08/hhai-2021_paper_46.pdf",
     "Training Intelligent Tutors on User Simulators Using Reinforcement Learning"],
    ["10.3233/FAIA230099", "https://doi.org/10.3233/FAIA230099",
     "Towards Robots at Meet Users’ Need for Explanation"],
    ["10.3233/FAIA240187", "https://doi.org/10.3233/FAIA240187",
     "EASY-AI: sEmantic And compoSable glYphs for representing AI systems"],
    ["10.3233/FAIA230097", "https://doi.org/10.3233/FAIA230097",
     "Co-Performing Music with AI: Real-Time Performance Control Using Speech and Gestures"],
    ["10.3233/FAIA230090", "https://doi.org/10.3233/FAIA230090",
     "Design of a Human-in-the-Loop Centered AI-Based Clinical Decision Support System for Professional Care Planning"],
    ["10.3233/FAIA230073", "https://doi.org/10.3233/FAIA230073",
     "Human Factors in Interactive Online Machine Learning"],
    ["10.3233/FAIA220187", "https://doi.org/10.3233/FAIA220187",
     "HyEnA: A Hybrid Method for Extracting Arguments from Opinions"],
    ["10.3233/FAIA230080", "https://doi.org/10.3233/FAIA230080",
     "Visualizing Deep Neural Networks with Topographic Activation Maps"],
    ["arxiv/2303.00866", "https://arxiv.org/abs/2303.00866",
     "A prototype hybrid prediction market for estimating replicability of published work"],
]

In [ ]:
lines = results_file_contents.split("\n")
result_lines = lines[1:]
print(lines[0])  # Header

In [ ]:
print(result_lines[1])

In [ ]:
# Classes used to model the results
class Scenario(BaseModel):
    main: str
    description: str
    contains: list[str]


class Task(BaseModel):
    main: str
    description: str
    hasActors: list[str]
    usedIn: list[str]


class IndividualActor(BaseModel):
    name: str
    type: str
    has: list[str]


class Actor(BaseModel):
    actors: list[IndividualActor]


class IndividualInformationProcessing(BaseModel):
    name: str
    description: str
    hasProcessingMethod: list[str]
    produces: list[str]


class InformationProcessing(BaseModel):
    information_processing: list[IndividualInformationProcessing]


class IndividualProcessingMethod(BaseModel):
    name: str
    type: str


class ProcessingMethod(BaseModel):
    methods: list[IndividualProcessingMethod]


class IndividualMetric(BaseModel):
    name: str
    type: str


class Metric(BaseModel):
    metrics: list[IndividualMetric]


class Result(BaseModel):
    paper_id: str
    paper_url: str
    paper_title: str
    scenario: Scenario
    task: Task
    actor: Actor
    information_processing: InformationProcessing
    processing_method: ProcessingMethod
    metric: Metric


def process_csv_part(line):
    line = line.replace("\"\"", "\"")

    line = ('{ "main": ' + line + '}').replace("\"\"", "\"")

    line_json = json.loads(line)
    return line_json


def process_csv_actor_part(line):
    line = line.replace("\"\"", "\"")

    line = ('{"actors": [ { "name": ' + line + '}]}').replace("\"\"", "\"")

    line_json = json.loads(line)

    return line_json


def process_csv_information_processing_part(line):
    line = line.replace("\"\"", "\"")

    line = ('{"information_processing": [ { "name": ' + line + '}]}').replace("\"\"", "\"")

    line_json = json.loads(line)
    return line_json


def process_csv_processing_method_part(line):
    line = line.replace("\"\"", "\"")

    line = ('{"methods": [ { "name": ' + line + '}]}').replace("\"\"", "\"")

    line_json = json.loads(line)
    return line_json


def process_csv_metric_part(line):
    line = line.replace("\"\"", "\"")

    line = ('{"metrics": [ { "name": ' + line + '}]}').replace("\"\"", "\"")

    line_json = json.loads(line)
    return line_json


results: list[Result] = []
for i, line in enumerate(result_lines):
    scenario_and_task_regex_string = open(
        Path(os.getcwd()).joinpath("scenario_and_task_regex.txt")).read()
    scenario_and_task_regex = re.compile(scenario_and_task_regex_string)
    scenario_and_task = scenario_and_task_regex.findall(line)

    scenario = process_csv_part(scenario_and_task[0])
    task = process_csv_part(scenario_and_task[1])

    rest_regex_string = open(
        Path(os.getcwd()).joinpath("rest_regex.txt")).read()
    rest_regex = re.compile(rest_regex_string)
    rest = rest_regex.findall(line)

    actor = process_csv_actor_part(rest[0])
    information_processing = process_csv_information_processing_part(rest[1])
    processing_method = process_csv_processing_method_part(rest[2])
    metric = process_csv_metric_part(rest[3])

    result = {
        "paper_id": paper_dois_and_titles[i][0],
        "paper_url": paper_dois_and_titles[i][1],
        "paper_title": paper_dois_and_titles[i][2],
        "scenario": scenario,
        "task": task,
        "actor": actor,
        "information_processing": information_processing,
        "processing_method": processing_method,
        "metric": metric
    }

    result = Result.model_validate(result)
    results.append(result)


In [ ]:
for result in results:
    print(result.model_dump_json(indent=4))

In [ ]:
# Conceptnet word embeddings for finding (out-of-vocabulary) concepts
vectors = {}
with open(Path(os.getcwd()).joinpath("./numberbatch-en.txt"), encoding='utf-8') as f:
    next(f)  # skip header
    for i, line in enumerate(f):
        parts = line.strip().split()
        word = parts[0]
        vec = np.array(list(map(float, parts[1:])))
        vectors[word] = vec
        # if i >= 1000:
        #     break

In [ ]:
# Load the transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
def load_conceptnet_words(vectors_dict):
    return [
        c.replace('_', ' ')
        for c in vectors_dict.keys()
        if c.startswith('/c/en/') or not c.startswith('/')
    ]


# Load ConceptNet entries
concepts = load_conceptnet_words(vectors)

# Embed concept names with the same model
concept_vecs = model.encode(concepts)

In [ ]:
CONCEPTNET_NAMESPACE = Namespace("https://api.conceptnet.io/c/en/")
DBPEDIA_NAMESPACE = Namespace("http://dbpedia.org/resource/")

concept_vecs = np.array(concept_vecs).astype('float32')
faiss.normalize_L2(concept_vecs)


def find_nearest_concepts(query: str):
    global concept_vecs

    query_vec = model.encode(query).reshape(1, -1)

    query_vec = query_vec.astype('float32')

    # Normalize for cosine similarity
    faiss.normalize_L2(query_vec)

    # Use FAISS due to the size of the dictionary
    index = faiss.IndexFlatIP(concept_vecs.shape[1])
    index.add(concept_vecs)

    k = 5
    scores, indices = index.search(query_vec, k)

    results: list[dict["concept" or "score", str]] = []
    for idx, score in zip(indices[0], scores[0]):
        if score > 0.8:
            results.append({
                "concept": URIRef(standardized_concept_uri("en", concepts[idx]),
                                  base=CONCEPTNET_NAMESPACE),
                "score": score
            })
            print(f"{word} ==>  {concepts[idx]} — {score:.4f}")

    return results

In [ ]:
# Uses reification to be able to add metadata to the relationships
def reificate_information_processing_for_actor(individual_actor_ref: URIRef | BNode,
                                               individual_information_processing_ref: URIRef) -> URIRef:
    reificated_information_processing_for_actor_ref = URIRef(
        individual_actor_ref + "_processing_information_" + individual_information_processing_ref,
        base=paper_ref)
    my_graph.add((
        reificated_information_processing_for_actor_ref,
        RDF.type,
        RDF.Statement
    ))
    my_graph.add((
        reificated_information_processing_for_actor_ref,
        RDF.subject,
        URIRef(individual_actor_ref)
    ))
    my_graph.add((
        reificated_information_processing_for_actor_ref,
        RDF.predicate,
        hi_processingInformationProperty_ref
    ))
    my_graph.add((
        reificated_information_processing_for_actor_ref,
        RDF.object,
        individual_information_processing_ref
    ))

    return reificated_information_processing_for_actor_ref


def reificate_interaction_for_scenario(individual_interaction_ref: URIRef,
                                       individual_scenario_ref: URIRef | BNode) -> URIRef:
    reficiated_interaction_for_scenario_ref = URIRef(
        individual_scenario_ref + "_interaction_" + individual_interaction_ref, base=paper_ref)
    my_graph.add((
        reficiated_interaction_for_scenario_ref,
        RDF.type,
        RDF.Statement
    ))
    my_graph.add((
        reficiated_interaction_for_scenario_ref,
        RDF.subject,
        URIRef(individual_scenario_ref)
    ))
    my_graph.add((
        reficiated_interaction_for_scenario_ref,
        RDF.predicate,
        hi_hasInteractionProperty_ref
    ))
    my_graph.add((
        reficiated_interaction_for_scenario_ref,
        RDF.object,
        individual_interaction_ref
    ))

    return reficiated_interaction_for_scenario_ref


def random_delay():
    delay = random.uniform(0, 0.2)  # random delay between 1 and 3 seconds
    print(f"Sleeping for {delay:.2f} seconds")
    time.sleep(delay)


def dbpedia_exists(uri: str):
    print(DONT_CHECK_FOR_URI_EXISTS)
    if DONT_CHECK_FOR_URI_EXISTS:
        return True

    random_delay()

    response = requests.head(uri, allow_redirects=True)
    return response.status_code == 200


def add_conceptnet_keywords(word: str, related_concepts_added_to_ref: URIRef):
    nearest_concepts = find_nearest_concepts(word)

    refs: list[URIRef] = []
    for concept in nearest_concepts:
        concept_name = concept["concept"].split("/")[-1].replace(" ", "_").capitalize()

        refs.append(concept["concept"])
        my_graph.add((
            related_concepts_added_to_ref,
            DCTERMS.subject,
            concept["concept"]
        ))

        dbpedia_link = URIRef(
            concept_name,
            base=DBPEDIA_NAMESPACE
        )
        print(dbpedia_link)
        if dbpedia_exists(dbpedia_link):
            my_graph.add((
                related_concepts_added_to_ref,
                DCTERMS.subject,
                dbpedia_link
            ))

        else:
            print(f"{dbpedia_link} does not exist")

    return refs


# Populate graph with instances
my_graph = Graph()
my_graph.remove((None, None, None))
my_graph.parse("./hi_ontology.ttl")

KGST_NAMESPACE = Namespace(
    "https://github.com/EliasLiinamaa/kgst_project_group_3/final_ontology_extension.ttl#")
my_graph.namespace_manager.bind("", KGST_NAMESPACE)

BIBO_NAMESPACE = Namespace("http://purl.org/ontology/bibo/")
my_graph.namespace_manager.bind("bibo", BIBO_NAMESPACE)

HI_NAMESPACE = Namespace(
    "http://www.semanticweb.org/vbr240/ontologies/2022/4/untitled-ontology-51/")
my_graph.namespace_manager.bind("hi", HI_NAMESPACE)

my_graph.namespace_manager.bind("conceptnet", CONCEPTNET_NAMESPACE)
my_graph.namespace_manager.bind("dbr", DBPEDIA_NAMESPACE)

bibo_AcademicArticleClass_ref = URIRef("AcademicArticle", base=BIBO_NAMESPACE)
kgst_HIAcademicArticleClass_ref = URIRef("HybridIntelligenceAcademicArticle", base=KGST_NAMESPACE)

kgst_examinesScenarioProperty_ref = URIRef("examinesScenario", base=KGST_NAMESPACE)
kgst_scenarioInProperty_ref = URIRef("scenarioIn", base=KGST_NAMESPACE)

hi_ScenarioClass_ref = URIRef("Scenario", base=HI_NAMESPACE)
hi_hasInteractionProperty_ref = URIRef("hasInteraction", base=HI_NAMESPACE)

hi_InteractionClass_ref = URIRef("Interaction", base=HI_NAMESPACE)
kgst_usesProcessProperty_ref = URIRef("usesProcess", base=KGST_NAMESPACE)
hi_interactingAgentProperty_ref = URIRef("interactingAgent", base=HI_NAMESPACE)

hi_ActorClass_ref = URIRef("Actor", base=HI_NAMESPACE)
hi_HumanClass_ref = URIRef("Human", base=HI_NAMESPACE)
hi_ArtificialAgentClass_ref = URIRef("ArtificialAgent", base=HI_NAMESPACE)
hi_inScenarioProperty_ref = URIRef("inScenario", base=HI_NAMESPACE)

hi_processingInformationProperty_ref = URIRef("processingInformation", base=HI_NAMESPACE)

hi_InformationProcessingClass_ref = URIRef("InformationProcessing", base=HI_NAMESPACE)
hi_informationMethodProperty_ref = URIRef("informationMethod", base=HI_NAMESPACE)

hi_ProcessingMethodClass_ref = URIRef("ProcessingMethod", base=HI_NAMESPACE)
# Under which dynamically generated sybclasses like `kgst:Symbolic` or `kgst:Statistical` etc.

kgst_metricProperty_ref = URIRef("metric", base=KGST_NAMESPACE)
kgst_MetricClass_ref = URIRef("Metric", base=KGST_NAMESPACE)
kgst_QuantitativeMetric_ref = URIRef("QuantitativeMetric", base=KGST_NAMESPACE)
kgst_QualitativeMetric_ref = URIRef("QualitativeMetric", base=KGST_NAMESPACE)

# ====================
# Research papers
my_graph.add((
    kgst_HIAcademicArticleClass_ref,
    RDFS.subClassOf,
    bibo_AcademicArticleClass_ref
))

my_graph.add((
    kgst_examinesScenarioProperty_ref,
    RDFS.range,
    hi_ScenarioClass_ref
))

# ====================
# Scenarios
my_graph.add((
    kgst_scenarioInProperty_ref,
    OWL.inverseOf,
    kgst_examinesScenarioProperty_ref
))

# ====================
# Interactions
my_graph.add((
    kgst_usesProcessProperty_ref,
    RDFS.domain,
    hi_InteractionClass_ref
))
my_graph.add((
    kgst_usesProcessProperty_ref,
    RDFS.range,
    hi_InformationProcessingClass_ref
))

# ====================
# Metrics
my_graph.add((
    kgst_QuantitativeMetric_ref,
    RDFS.subClassOf,
    kgst_MetricClass_ref
))
my_graph.add((
    kgst_QualitativeMetric_ref,
    RDFS.subClassOf,
    kgst_MetricClass_ref
))
my_graph.add((
    kgst_metricProperty_ref,
    RDFS.range,
    kgst_MetricClass_ref
))

# Go through each paper
for paper in results:
    paper_ref = URIRef(paper.paper_id, base=KGST_NAMESPACE)

    # Add paper to the graph
    my_graph.add((
        paper_ref,
        RDF.type,
        kgst_HIAcademicArticleClass_ref
    ))

    # Basic details
    my_graph.add((
        paper_ref,
        DCTERMS.title,
        Literal(paper.paper_title)
    ))
    my_graph.add((
        paper_ref,
        RDFS.label,
        Literal(paper.paper_title)
    ))
    my_graph.add((
        paper_ref,
        OWL.sameAs,
        URIRef(paper.paper_url)
    ))

    # Scenario
    individual_scenario_ref = URIRef(f"_scenario_{paper.scenario.main.replace(' ', '_')}",
                                     base=paper_ref)
    my_graph.add((
        individual_scenario_ref,
        RDF.type,
        hi_ScenarioClass_ref
    ))
    my_graph.add((
        paper_ref,
        kgst_examinesScenarioProperty_ref,
        individual_scenario_ref
    ))
    my_graph.add((
        individual_scenario_ref,
        RDFS.label,
        Literal(paper.scenario.main)
    ))
    my_graph.add((
        individual_scenario_ref,
        DCTERMS.description,
        Literal(paper.scenario.description)
    ))
    add_conceptnet_keywords(paper.scenario.main, individual_scenario_ref)

    # Add interactions
    specific_interactions: dict[URIRef, URIRef] = {}
    for interaction in paper.scenario.contains:
        individual_interaction_ref = URIRef(
            interaction.replace(" ", "_"), base=KGST_NAMESPACE)
        my_graph.add((
            individual_interaction_ref,
            RDFS.label,
            Literal(interaction)
        ))

        reificated_interaction_for_scenario_ref = reificate_interaction_for_scenario(
            individual_interaction_ref, individual_scenario_ref)
        specific_interactions[individual_interaction_ref] = reificated_interaction_for_scenario_ref

        add_conceptnet_keywords(interaction, individual_interaction_ref)

    # Task that may or may not have already been included in paper.scenario.contains
    individual_interaction_ref = URIRef(
        paper.task.main.replace(" ", "_"), base=KGST_NAMESPACE)

    reificated_interaction_for_scenario_ref: URIRef
    if individual_interaction_ref in specific_interactions:
        reificated_interaction_for_scenario_ref = specific_interactions[
            individual_interaction_ref]
    else:
        reificated_interaction_for_scenario_ref = reificate_interaction_for_scenario(
            individual_interaction_ref, individual_scenario_ref)

    my_graph.add((
        reificated_interaction_for_scenario_ref,
        RDFS.label,
        Literal(paper.task.main)
    ))

    # Add actors
    for actor in paper.task.hasActors:
        individual_actor_ref = URIRef("_actor_" + actor.replace(" ", "_"),
                                      base=paper_ref)
        my_graph.add((
            reificated_interaction_for_scenario_ref,
            hi_interactingAgentProperty_ref,
            individual_actor_ref
        ))

        add_conceptnet_keywords(actor, individual_actor_ref)

    # Add information processing
    for process in paper.task.usedIn:
        individual_information_processing_ref = URIRef(
            process.replace(" ", "_"),
            base=KGST_NAMESPACE)
        my_graph.add((
            reificated_interaction_for_scenario_ref,
            kgst_usesProcessProperty_ref,
            individual_information_processing_ref
        ))
        my_graph.add((
            individual_information_processing_ref,
            RDF.type,
            hi_InformationProcessingClass_ref
        ))
        add_conceptnet_keywords(process, individual_information_processing_ref)

    # Process additional details for actors
    actor_information_processings: dict[
        URIRef, URIRef] = cast(dict[URIRef, URIRef],
                               {})  # "individual_information_processing_uri_ref" : "reificated_information_processing_for_actor_ref"
    for actor in paper.actor.actors:
        individual_actor_ref = URIRef("_actor_" + actor.name.replace(" ", "_"),
                                      base=paper_ref)

        add_conceptnet_keywords(actor.name, individual_actor_ref)

        my_graph.add((
            individual_actor_ref,
            hi_inScenarioProperty_ref,
            individual_scenario_ref,
        ))

        assert actor.type in ["Human", "Artificial Agent"]
        my_graph.add((
            individual_actor_ref,
            RDF.type,
            hi_HumanClass_ref if actor.type.lower() == "human" else hi_ArtificialAgentClass_ref
        ))  # Shouldn't set range of schema participant

        # Add actor's information processing
        for actor_information_processing in actor.has:
            individual_information_processing_uri_ref = URIRef(
                actor_information_processing.replace(" ", "_"), base=KGST_NAMESPACE)
            reificated_information_processing_for_actor_ref: URIRef = reificate_information_processing_for_actor(
                individual_actor_ref,
                individual_information_processing_uri_ref)

            actor_information_processings[
                individual_information_processing_uri_ref] = reificated_information_processing_for_actor_ref

        add_conceptnet_keywords(actor.name, individual_actor_ref)

    # Process additional details for information processing
    metrics: list[URIRef] = []
    for information_processing in paper.information_processing.information_processing:
        individual_information_processing_ref = URIRef(
            information_processing.name.replace(" ", "_"),
            base=KGST_NAMESPACE)
        my_graph.add((
            individual_information_processing_ref,
            SKOS.definition,  # "A formal, human-readable definition of a skos:Concept"
            Literal(information_processing.description)
        ))

        add_conceptnet_keywords(information_processing.name,
                                individual_information_processing_ref)

        my_graph.add((
            individual_information_processing_ref,
            RDFS.label,
            Literal(information_processing.name)
        ))

        reificated_information_processing_for_actor_ref: URIRef
        if individual_information_processing_ref in actor_information_processings.keys():
            reificated_information_processing_for_actor_ref = actor_information_processings[
                individual_information_processing_ref]
        else:
            print(individual_information_processing_ref, "has NOT been assigned to an actor")
            reificated_information_processing_for_actor_ref = reificate_information_processing_for_actor(
                BNode(),
                individual_information_processing_ref)

        # Add processing methods for the information processing
        for processing_method in information_processing.hasProcessingMethod:
            individual_processing_method_ref = URIRef(processing_method.replace(" ", "_"),
                                                      base=KGST_NAMESPACE)
            my_graph.add((
                reificated_information_processing_for_actor_ref,
                hi_informationMethodProperty_ref,
                individual_processing_method_ref
            ))
            my_graph.add((
                individual_processing_method_ref,
                RDF.type,
                hi_ProcessingMethodClass_ref
            ))
            my_graph.add((
                individual_processing_method_ref,
                RDFS.label,
                Literal(processing_method)
            ))
            add_conceptnet_keywords(processing_method,
                                    individual_processing_method_ref)

        # Add metrics for the information processing
        for metric in information_processing.produces:
            individual_metric_ref = URIRef(metric.replace(" ", "_"), base=KGST_NAMESPACE)
            my_graph.add((
                reificated_information_processing_for_actor_ref,
                kgst_metricProperty_ref,
                individual_metric_ref
            ))
            my_graph.add((
                individual_metric_ref,
                RDF.type,
                kgst_MetricClass_ref
            ))

        add_conceptnet_keywords(information_processing.name,
                                individual_information_processing_ref)

    # Process additional details for processing methods
    for processing_method in paper.processing_method.methods:
        individual_processing_method_class_ref = URIRef(
            processing_method.type.replace(" ", "_"), base=KGST_NAMESPACE)
        my_graph.add((
            individual_processing_method_class_ref,
            RDFS.subClassOf,
            hi_ProcessingMethodClass_ref
        ))

        individual_processing_method_ref = URIRef(processing_method.name.replace(" ", "_"),
                                                  base=KGST_NAMESPACE)
        my_graph.add((
            individual_processing_method_ref,
            RDF.type,
            individual_processing_method_class_ref
        ))
        my_graph.add((
            individual_processing_method_ref,
            RDFS.label,
            Literal(processing_method.name)
        ))

        add_conceptnet_keywords(processing_method.name, individual_processing_method_ref)

    # Process additional details for metrics
    for metric in paper.metric.metrics:
        individual_metric_ref = URIRef(metric.name.replace(" ", "_"), base=KGST_NAMESPACE)
        my_graph.add((
            individual_metric_ref,
            RDFS.label,
            Literal(metric.name)
        ))
        assert metric.type in ["Qualitative", "Quantitative"]
        my_graph.add((
            individual_metric_ref,
            RDF.type,
            kgst_QualitativeMetric_ref if metric.type.lower() == "qualitative" else kgst_QuantitativeMetric_ref
        ))
        add_conceptnet_keywords(metric.name, individual_metric_ref)

In [ ]:
print(my_graph.serialize(format="turtle"))
my_graph.serialize("./final_ontology_extension.ttl", format="turtle")
my_graph.serialize("./final_ontology_extension.nt", encoding="utf-8", format="ntriples")

In [ ]:
print(len(my_graph))
owlrl.DeductiveClosure(owlrl.OWLRL_Semantics).expand(my_graph)
print(len(my_graph))
my_graph.serialize("./final_ontology_extension_inferred.ttl", format="turtle")

In [ ]:
# Check for consistency

world = owlready2.World()
world.get_ontology("./final_ontology_extension.nt", ).load()

owlready2.sync_reasoner()

inconsistent_classes = list(world.inconsistent_classes())

if inconsistent_classes:
    print("Inconsistent classes found:")
    for cls in inconsistent_classes:
        print(cls)
else:
    print("Ontology is consistent!")